# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
from datetime import timedelta
import papermill as pm
from nord_h2ub_dropdown_functions import *

### Base Parameters:

In [ ]:
combined_dropdowns, dropdowns = create_combined_dropdowns()
display(combined_dropdowns)

In [2]:
# Example usage
combined_tabs, dropdowns = create_combined_dropdowns_tabs()
display(combined_tabs)

In [ ]:
#get the values after setting the inputs by the user
values = get_dropdown_values(dropdowns)

In [ ]:
# Create the combined multiple choice fields and display them
combined_multiple_choices, selected_options = create_combined_multiple_choices()
display(combined_multiple_choices)

In [ ]:
#required reports
report_name = 'Report'

In [ ]:
#financial parameters for the calculations
#when using the district heating as a revenue stream, the price cap is the upper limit in Denmark
#by using this parameter this can be adjusted to start with other than the max value
share_of_dh_price_cap = 0.5
price_level_power = 1
power_price_variance = 1

In [ ]:
#scenario
scenario = 'Base'

#model name
model_name = 'toy'

#stochastics
stochastic_scenario = "realisation"
stochastic_structure = "deterministic"

In [ ]:
#define parameters for the use of the roll forward (rolling horizon) approach
#set the number of slices within the modelling horizon
roll_forward_use = True
num_slices = 12
# Create DatetimeIndex for the range of dates
start_date = pd.Timestamp(str(year_dropdown.value) + '-01-01 00:00:00')
end_date = pd.Timestamp(str(year_dropdown.value) + '-12-31 23:00:00')
datetime_index = pd.date_range(start=start_date, end=end_date, freq=frequency_dropdown.value)
# Calculate the number of steps within the horizon
num_steps = len(datetime_index)
# Check if the number of slices can be used
# Find the largest integer divisor that fulfils the condition
for i in range(num_slices, 0, -1):
    if num_steps % i == 0:
        roll_forward_size = num_steps // i
        used_slices = i
        break
else:
    print("Cannot divide the number of steps into any integer slices. Please choose a different number of slices.")
    exit()
# Check if num_slices matches the used_slices
if num_slices != used_slices:
    print("\033[91mWARNING:\033[0m The specified number of slices (", num_slices, ")",
          "does not match the final division factor (", used_slices, ").",
         "\n The calculation uses the factor: ",used_slices, ".")


# Print warning message in red
print("\033[91mWARNING:\033[0m Please control if all the parameters are set correctly")

In [ ]:
#temporal block, do not change
if dropdowns['frequency'].value == '1h':
    temporal_block = 'hourly'
elif dropdowns['frequency'].value == '1D':
    temporal_block = 'daily'
elif dropdowns['frequency'].value == '1W':
    temporal_block = 'weekly'
elif dropdowns['frequency'].value == '1M':
    temporal_block = 'monthly'
elif dropdowns['frequency'].value == '1Q':
    temporal_block = 'quarterly'
else: 
    temporal_block = 'yearly'

### Dictionary:

In [ ]:
base_parameters = {
    'year': dropdowns['year'].value,
    'start_date': (pd.Timestamp(str(dropdowns['year'].value) + '-01-01 00:00:00')).strftime('%Y-%m-%d %H:%M:%S'),
    'end_date': (pd.Timestamp(str(dropdowns['year'].value) + '-12-31 23:00:00')).strftime('%Y-%m-%d %H:%M:%S'),
    'area': dropdowns['price_zone'].value,
    'product': dropdowns['product'].value,
    'scenario': scenario,
    'frequency': dropdowns['frequency'].value,
    'model_name': model_name,
    'temporal_block': temporal_block,
    'stochastic_scenario': stochastic_scenario,
    'stochastic_structure': stochastic_structure,
    'report_name': report_name,
    'reports': selected_options,
    'electrolyzer_type': electrolysis_value = dropdowns['electrolysis'].value,
    'des_segments_electrolyzer': ,
    'share_of_dh_price_cap': share_of_dh_price_cap,
    'price_level_power': price_level_power,
    'power_price_variance': power_price_variance,
    'roll_forward_use': roll_forward_use,
    'roll_forward_size': roll_forward_size,
    'num_slices': num_slices,
    'num_steps': num_steps
}

### Execute standard routine:

In [ ]:
pm.execute_notebook(
    'data_preparation_nord_h2ub_spine.ipynb',
    'please_ignore.ipynb',
    parameters=base_parameters
)